In [ ]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer

# Load the dataset
file_path = '/content/medical_review.csv'
data = pd.read_csv(file_path)

# Step 1: Data Cleaning
# Renaming columns for consistency
data.columns = data.columns.str.strip().str.replace(' ', '_').str.lower()

# Handling missing values
for col in ['diagnosis_category', 'diagnosis_sub_category', 'treatment_category',
            'treatment_sub_category', 'age_range', 'patient_gender']:
    data[col] = data[col].fillna('Unknown')

data['findings'] = data['findings'].fillna('No findings provided')

# Removing duplicates
data = data.drop_duplicates()

# Step 2: Standardizing Text Columns
text_columns = ['diagnosis_category', 'diagnosis_sub_category', 'treatment_category',
                'treatment_sub_category', 'determination', 'type', 'age_range',
                'patient_gender', 'findings']

for col in text_columns:
    data[col] = data[col].str.strip().str.lower()

# Step 3: Univariate Analysis
categorical_columns = ['diagnosis_category', 'treatment_category', 'determination', 'type', 'age_range', 'patient_gender']

for col in categorical_columns:
    plt.figure(figsize=(10, 6))
    data[col].value_counts().plot(kind='bar', title=f'Distribution of {col.capitalize()}', ylabel='Count', xlabel=col.capitalize())
    plt.xticks(rotation=45)
    plt.show()

# Step 4: Bivariate Analysis
# Diagnosis Category vs Determination
plt.figure(figsize=(14, 7))
sns.countplot(data=data, x='diagnosis_category', hue='determination', order=data['diagnosis_category'].value_counts().index[:10])
plt.title('Diagnosis Category vs Determination')
plt.xticks(rotation=45)
plt.ylabel('Count')
plt.xlabel('Diagnosis Category')
plt.legend(title='Determination', loc='upper right')
plt.show()

# Age Range vs Patient Gender
plt.figure(figsize=(12, 6))
sns.countplot(data=data, x='age_range', hue='patient_gender', order=data['age_range'].value_counts().index)
plt.title('Age Range vs Patient Gender')
plt.xticks(rotation=45)
plt.ylabel('Count')
plt.xlabel('Age Range')
plt.legend(title='Patient Gender', loc='upper right')
plt.show()

# Step 5: Text Analysis
# Tokenize and count word frequencies in the 'findings' column
vectorizer = CountVectorizer(stop_words='english', max_features=20)
word_counts = vectorizer.fit_transform(data['findings'])

# Summarizing most frequent words
word_frequency = pd.DataFrame({
    'word': vectorizer.get_feature_names_out(),
    'count': word_counts.toarray().sum(axis=0)
}).sort_values(by='count', ascending=False)

# Visualizing the most frequent words
plt.figure(figsize=(10, 6))
word_frequency.set_index('word').plot(kind='bar', legend=False, figsize=(10, 6))
plt.title('Most Frequent Words in Findings')
plt.ylabel('Frequency')
plt.xlabel('Word')
plt.xticks(rotation=45)
plt.show()

# Save the cleaned and processed dataset
processed_file_path = 'processed_medical_review.csv'
data.to_csv(processed_file_path, index=False)

print(f"Processed dataset saved to: {processed_file_path}")


In [ ]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# 1. Memuat dataset
file_path = '/content/processed_medical_review.csv'  # Ganti dengan path file Anda
data = pd.read_csv(file_path)

# 2. Analisis frekuensi kata untuk menemukan kata kunci yang sering muncul
vectorizer = CountVectorizer(stop_words='english', max_features=50)
X = vectorizer.fit_transform(data['findings'].dropna())
word_freq = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
word_freq_sum = word_freq.sum().sort_values(ascending=False)
print("Top 10 kata berdasarkan frekuensi:")
print(word_freq_sum.head(10))

# 3. Menggunakan TF-IDF untuk mendapatkan kata kunci penting
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=50)
X_tfidf = tfidf_vectorizer.fit_transform(data['findings'].dropna())
tfidf_terms = pd.DataFrame(X_tfidf.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
tfidf_terms_sum = tfidf_terms.sum().sort_values(ascending=False)
print("Top 10 kata berdasarkan skor TF-IDF:")
print(tfidf_terms_sum.head(10))

# 4. Ekstraksi kata kunci pengobatan dari kolom "findings"
treatment_keywords = [
    'medication', 'drug', 'therapy', 'prescription', 'surgery',
    'radiation', 'chemotherapy', 'hospitalization', 'procedure', 'diagnostic', 'therapy'
]

def extract_treatment_terms(text, keywords):
    found_terms = []
    for keyword in keywords:
        if re.search(r'\b' + keyword + r'\b', text, re.IGNORECASE):
            found_terms.append(keyword)
    return ', '.join(found_terms) if found_terms else None

# Terapkan ekstraksi pengobatan pada kolom 'findings'
data['extracted_treatments'] = data['findings'].apply(lambda x: extract_treatment_terms(str(x), treatment_keywords))

# 5. Menampilkan hasil ekstraksi pengobatan
print("Hasil ekstraksi pengobatan:")
print(data[['reference_id', 'findings', 'extracted_treatments']].head())

# 6. Membuat tabel pivot untuk menganalisis hubungan antara kategori diagnosis dan pengobatan
pivot_treatment_diagnosis = pd.crosstab(data['diagnosis_category'], data['extracted_treatments'])

# 7. Menampilkan hasil analisis hubungan diagnosis dengan pengobatan
print("Hubungan antara diagnosis dan pengobatan:")
print(pivot_treatment_diagnosis)

# 8. Menyimpan hasil analisis dalam file CSV (opsional)
pivot_treatment_diagnosis.to_csv('/content/treatment_diagnosis_summary.csv')  # Ganti dengan path file tujuan

# --- EDA (Exploratory Data Analysis) ---

# 9. Visualisasi distribusi kata berdasarkan frekuensi (Word Frequency)
plt.figure(figsize=(10, 6))
word_freq_sum.head(20).plot(kind='bar', color='skyblue')
plt.title('Top 20 Kata Berdasarkan Frekuensi')
plt.xlabel('Kata')
plt.ylabel('Frekuensi')
plt.xticks(rotation=45, ha='right')
plt.show()

# 10. Visualisasi kata penting berdasarkan skor TF-IDF
plt.figure(figsize=(10, 6))
tfidf_terms_sum.head(20).plot(kind='bar', color='lightcoral')
plt.title('Top 20 Kata Berdasarkan Skor TF-IDF')
plt.xlabel('Kata')
plt.ylabel('Skor TF-IDF')
plt.xticks(rotation=45, ha='right')
plt.show()

# 11. Visualisasi distribusi kategori diagnosis
plt.figure(figsize=(10, 6))
sns.countplot(data=data, x='diagnosis_category', palette='Set2')
plt.title('Distribusi Kategori Diagnosis')
plt.xlabel('Kategori Diagnosis')
plt.ylabel('Jumlah Kasus')
plt.xticks(rotation=45, ha='right')
plt.show()

# 12. Visualisasi hubungan diagnosis dengan pengobatan (treatment)
# Menghitung frekuensi pengobatan per diagnosis
treatment_counts = pivot_treatment_diagnosis.sum(axis=1).sort_values(ascending=False)

plt.figure(figsize=(10, 6))
treatment_counts.head(20).plot(kind='bar', color='lightgreen')
plt.title('Top 20 Diagnosis Berdasarkan Jumlah Pengobatan')
plt.xlabel('Diagnosis')
plt.ylabel('Jumlah Pengobatan')
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import matplotlib.pyplot as plt
import seaborn as sns

# Load dataset
file_path = '/content/processed_medical_review.csv'  # Path in Google Colab
data = pd.read_csv(file_path)

# 1. **Text Analysis**: Word Frequency Analysis
vectorizer = CountVectorizer(stop_words='english', max_features=50)  # Limit to 50 most common words
X = vectorizer.fit_transform(data['findings'].dropna())
word_freq = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
word_freq_sum = word_freq.sum().sort_values(ascending=False)
# Display the top 10 words by frequency
print("Top Word Frequency:")
print(word_freq_sum.head(10))

# 2. **TF-IDF Analysis** (Identifying Key Phrases)
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=50)
X_tfidf = tfidf_vectorizer.fit_transform(data['findings'].dropna())
tfidf_terms = pd.DataFrame(X_tfidf.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
tfidf_terms_sum = tfidf_terms.sum().sort_values(ascending=False)
# Display the top 10 TF-IDF terms
print("Top TF-IDF Terms:")
print(tfidf_terms_sum.head(10))

# 3. **Extract Treatment Terms** from 'findings' column
treatment_keywords = ['medication', 'drug', 'therapy', 'prescription', 'surgery', 'radiation', 'chemotherapy', 'hospitalization', 'procedure', 'diagnostic']

def extract_treatment_terms(text, keywords):
    found_terms = []
    for keyword in keywords:
        if re.search(r'\b' + keyword + r'\b', text, re.IGNORECASE):
            found_terms.append(keyword)
    return ', '.join(found_terms) if found_terms else None

data['extracted_treatments'] = data['findings'].apply(lambda x: extract_treatment_terms(str(x), treatment_keywords))
# Display the first few rows with the extracted treatments
print("Extracted Treatments:")
print(data[['reference_id', 'findings', 'extracted_treatments']].head())

# 4. **Relationship between Diagnosis and Treatments**
pivot_treatment_diagnosis = pd.crosstab(data['diagnosis_category'], data['extracted_treatments'])
# Display the pivot table for treatment and diagnosis
print("Treatment and Diagnosis Summary:")
print(pivot_treatment_diagnosis)

# 5. **Analyzing Determination (Upheld vs. Overturned) based on Diagnosis and Treatment**
filtered_determination = pd.crosstab(
    data['determination'],
    [data['diagnosis_category'], data['extracted_treatments']],
    margins=True,  # Add totals to the table
    margins_name="Total"
)
# Display the determination distribution table
print("Diagnosis and Treatment Distribution by Determination:")
print(filtered_determination)

# -- EDA Section: Exploratory Data Analysis --

# Basic Descriptive Statistics
print("\nBasic Descriptive Statistics:")
print(data.describe(include='all'))

# Visualize Distribution of Determination (Upheld vs. Overturned)
plt.figure(figsize=(6, 4))
sns.countplot(x='determination', data=data, palette='Set2')
plt.title('Distribution of Determination (Upheld vs. Overturned)')
plt.xlabel('Determination')
plt.ylabel('Count')
plt.show()

# Visualize Relationship Between Determination and Diagnosis Category
plt.figure(figsize=(10, 6))
sns.countplot(x='diagnosis_category', hue='determination', data=data, palette='Set1')
plt.title('Determination Distribution by Diagnosis Category')
plt.xlabel('Diagnosis Category')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

# 1. Load the dataset
file_path = '/content/medicine.csv'  # Ganti dengan path file Anda
data = pd.read_csv(file_path)

# 2. Data Cleaning: Strip whitespaces and standardize text columns
data['Drug_Name'] = data['Drug_Name'].str.strip().str.lower()  # Standardize Drug Name (upper case)
data['Reason'] = data['Reason'].str.strip().str.title()  # Standardize Reason (title case)
data['Description'] = data['Description'].str.strip().str.lower()  # Standardize Description (lower case)

# 3. Remove duplicates
data_cleaned = data.drop_duplicates(subset=['Drug_Name', 'Reason', 'Description'])

# 4. Check for missing values (None expected in this case)
missing_values = data_cleaned.isnull().sum()
print(f"Missing values: \n{missing_values}")

# 5. EDA - Distribution of Drug Names (Top 10)
top_drugs = data_cleaned['Drug_Name'].value_counts().head(10)

# 6. EDA - Distribution of Reason (Top 10)
top_reasons = data_cleaned['Reason'].value_counts().head(10)

# 7. EDA - Length of Descriptions (word count per description)
data_cleaned['Description_Length'] = data_cleaned['Description'].apply(lambda x: len(x.split()))

# 8. Visualizations
plt.figure(figsize=(10, 6))

# Drug Name Distribution
plt.subplot(1, 2, 1)
sns.barplot(x=top_drugs.values, y=top_drugs.index, palette="viridis")
plt.title('Top 10 Most Common Drug Names')
plt.xlabel('Frequency')
plt.ylabel('Drug Name')

# Reason Distribution
plt.subplot(1, 2, 2)
sns.barplot(x=top_reasons.values, y=top_reasons.index, palette="plasma")
plt.title('Top 10 Most Common Reasons')
plt.xlabel('Frequency')
plt.ylabel('Reason')

plt.tight_layout()
plt.show()

# Word Cloud of Most Frequent Terms in Description
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(' '.join(data_cleaned['Description']))

# Display the word cloud
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud of Description Texts')
plt.show()

# Distribution of Description Lengths
plt.figure(figsize=(8, 6))
sns.histplot(data_cleaned['Description_Length'], kde=True, color='blue', bins=20)
plt.title('Distribution of Description Lengths (Word Count)')
plt.xlabel('Word Count')
plt.ylabel('Frequency')
plt.show()

# 9. Save the cleaned dataset to a CSV file
cleaned_file_path = '/content/cleaned_medicine_data.csv'  # Ganti dengan path tempat Anda ingin menyimpan file
data_cleaned.to_csv(cleaned_file_path, index=False)

print(f"Cleaned data saved to: {cleaned_file_path}")

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# 1. Membaca dataset yang sudah diupload
file_path = '/content/cleaned_medicine_data.csv'  # Sesuaikan dengan path file
data = pd.read_csv(file_path)

# 2. Feature Engineering

# 2.1 Panjang Nama Obat
data['Drug_Name_Length'] = data['Drug_Name'].apply(len)

# 2.2 Jumlah Kata dalam Nama Obat
data['Drug_Name_Word_Count'] = data['Drug_Name'].apply(lambda x: len(x.split()))

# 2.3 Jumlah Kata dalam Deskripsi
data['Description_Length'] = data['Description'].apply(lambda x: len(x.split()))

# 2.4 Kategori Alasan (contoh: "Acne", "Pain", atau lainnya)
data['Reason_Category'] = data['Reason'].apply(lambda x: 'Acne' if 'Acne' in x else ('Pain' if 'Pain' in x else 'Other'))

# 2.5 Kehadiran Kata "Acne" dalam Deskripsi
data['Contains_Acne'] = data['Description'].apply(lambda x: 1 if 'acne' in x.lower() else 0)

# 2.7 Rasio Nama Obat terhadap Deskripsi
data['Name_to_Description_Ratio'] = data['Drug_Name_Length'] / data['Description_Length']

# Tampilkan beberapa baris pertama untuk melihat fitur baru yang ditambahkan
print(data.head())

# 3. Eksplorasi Data (EDA) dan Visualisasi

# Set up a large plot size
plt.figure(figsize=(16, 10))

# 3.1 Distribusi Panjang Nama Obat
plt.subplot(2, 3, 1)
sns.histplot(data['Drug_Name_Length'], kde=True, color='blue', bins=20)
plt.title('Distribusi Panjang Nama Obat')
plt.xlabel('Panjang Nama Obat')
plt.ylabel('Frekuensi')

# 3.2 Distribusi Jumlah Kata dalam Nama Obat
plt.subplot(2, 3, 2)
sns.histplot(data['Drug_Name_Word_Count'], kde=True, color='green', bins=20)
plt.title('Distribusi Jumlah Kata dalam Nama Obat')
plt.xlabel('Jumlah Kata dalam Nama Obat')
plt.ylabel('Frekuensi')

# 3.3 Distribusi Jumlah Kata dalam Deskripsi
plt.subplot(2, 3, 3)
sns.histplot(data['Description_Length'], kde=True, color='orange', bins=20)
plt.title('Distribusi Jumlah Kata dalam Deskripsi')
plt.xlabel('Jumlah Kata dalam Deskripsi')
plt.ylabel('Frekuensi')

# 3.4 Distribusi Rasio Nama Obat terhadap Deskripsi
plt.subplot(2, 3, 4)
sns.histplot(data['Name_to_Description_Ratio'], kde=True, color='purple', bins=20)
plt.title('Distribusi Rasio Nama Obat terhadap Deskripsi')
plt.xlabel('Rasio Nama Obat terhadap Deskripsi')
plt.ylabel('Frekuensi')

# 3.6 Distribusi Kategori Alasan
plt.subplot(2, 3, 5)
sns.countplot(x='Reason_Category', data=data, palette='Set2')
plt.title('Distribusi Kategori Alasan Penggunaan Obat')
plt.xlabel('Kategori Alasan')
plt.ylabel('Jumlah')

plt.tight_layout()
plt.show()

# 3.7 Heatmap untuk Korelasi Fitur Numerik
corr_matrix = data[['Drug_Name_Length', 'Drug_Name_Word_Count', 'Description_Length', 'Contains_Acne', 'Name_to_Description_Ratio']].corr()
plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title('Heatmap Korelasi Fitur Numerik')
plt.show()

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load datasets
processed_medical_review_path = '/content/processed_medical_review.csv'
cleaned_medicine_data_path = '/content/cleaned_medicine_data.csv'

processed_medical_review_df = pd.read_csv(processed_medical_review_path)
cleaned_medicine_data_df = pd.read_csv(cleaned_medicine_data_path)

# Initialize TF-IDF Vectorizer
vectorizer = TfidfVectorizer()

# Fit the TF-IDF vectorizer on the 'diagnosis_sub_category' and 'Reason' columns
tfidf_diagnosis = vectorizer.fit_transform(processed_medical_review_df['diagnosis_sub_category'])
tfidf_reasons = vectorizer.transform(cleaned_medicine_data_df['Reason'])

# Calculate cosine similarity between all diagnosis sub-categories and reasons
cosine_similarities = cosine_similarity(tfidf_diagnosis, tfidf_reasons)

# Now, let's find the maximum similarity per diagnosis sub-category
matching_values = []
for i, diagnosis in enumerate(processed_medical_review_df['diagnosis_sub_category']):
    # Get the index of the most similar reason
    best_match_idx = cosine_similarities[i].argmax()

    # Get the similarity score
    best_match_score = cosine_similarities[i][best_match_idx]

    # Only add if the similarity score is above a threshold (e.g., 0.2)
    if best_match_score > 0.2:  # You can adjust this threshold as needed
        matching_values.append((diagnosis, cleaned_medicine_data_df.iloc[best_match_idx]['Reason'], best_match_score))

# Display the matching diagnosis and reasons (with similarity score)
matching_values

# Function to find matching medicines based on user input
def find_matching_medicines(diagnosis_input):
    # Match input to reason column
    matching_reasons = cleaned_medicine_data_df[cleaned_medicine_data_df['Reason'].str.lower() == diagnosis_input.lower()]

    # Check if any matching reasons found
    if not matching_reasons.empty:
        # Return list of drug names that match the diagnosis
        return matching_reasons['Drug_Name'].tolist()
    else:
        return []

# Simulate user input for diagnosis
def get_user_input_with_matching_values():
    # Simulate a user input (you can replace this part with actual input for real testing)
    diagnosis_input = input("Please enter your diagnosis from the list above: ")  # For real use, use input() here

    print(f"User input: {diagnosis_input}")

    # Find matching medicines for the given input
    matching_medicines = find_matching_medicines(diagnosis_input)

    if matching_medicines:
        print("\nMedicines that match your diagnosis:")
        for i, medicine in enumerate(matching_medicines, 1):
            print(f"{i}. {medicine}")
    else:
        print("No matching drugs found for your diagnosis.")

# Run the simulation
get_user_input_with_matching_values()